In [6]:
from utility_dfcounter import *

## 1. count nominal

In [2]:
def countDataFrames(variation=""):
    labels  = ["trigger","usetag","acc","accVar","nmcbg","nmcbgVar","nfake","nfakeVar","ndata","ndataVar"]
    records = []
    
    for trigger in ["mu","e"]:
        for usetag in ["1b","2b"]:
            
            print( "counting "+trigger+usetag + " ...")

            counter = DFCounter(trigger,usetag)
            counter.setVariation(variation)

            acc,accVar = counter.returnAcc()
            nmcbg,nmcbgVar = counter.returnNMCbg()
            nfake,nfakeVar = counter.returnNFake()
            ndata,ndataVar = counter.returnNData()
            records.append( (trigger,usetag,acc,accVar,nmcbg,nmcbgVar,nfake,nfakeVar,ndata,ndataVar) )

    df = pd.DataFrame.from_records(records, columns=labels)
    df.to_pickle( common.getBaseDirectory() + "data/counts/count_{}.pkl".format(variation))
    print( "counting finished!")
    
def countDataFrames_selection(variation=""):
    labels  = ["selection","nbjet","acc","accVar","nmcbg","nmcbgVar","nfake","nfakeVar","ndata","ndataVar"]
    records = []
    
    for selection in ["mumu","ee"]:
        for nbjet in ["==1",">1"]:

            counter = DFCounter_selection(selection,nbjet)
            counter.setVariation(variation)

            acc,accVar = counter.returnAcc()
            nmcbg,nmcbgVar = counter.returnNMCbg()
            nfake,nfakeVar = np.zeros(2), np.zeros(2)
            ndata,ndataVar = counter.returnNData()
            records.append( (selection,nbjet,acc,accVar,nmcbg,nmcbgVar,nfake,nfakeVar,ndata,ndataVar) )

    df = pd.DataFrame.from_records(records, columns=labels)
    df.to_pickle( common.getBaseDirectory() + "data/counts/DNNcount_{}.pkl".format(variation))
    print( "counting finished!")

In [3]:
#countDataFrames()

## 2. count with variation

In [4]:
import multiprocessing as mp

def runCountDataFrames(vlist):
    processes = []
    for v in vlist:
        processes.append(mp.Process(target=countDataFrames,args=(v,)))
    for ps in processes:
        ps.start()
    for ps in processes:
        ps.join()

In [5]:
# runCountDataFrames(['EPtDown','MuPtDown','TauPtDown',"TTXSUp","TWXSUp","",
#                     "JESUp","JESDown","JERUp","JERDown",
#                     "BTagUp","BTagDown","MistagUp","MistagDown"]) 

# runCountDataFrames(["RenormUp","RenormDown","FactorUp","FactorDown","PDFUp","PDFDown",
#                     'FSRUp','FSRDown','ISRUp','ISRDown','UEUp','UEDown','MEPSUp','MEPSDown']) 

# runCountDataFrames(['FSRUp','FSRDown','ISRUp','ISRDown']) # ttTheory
# runCountDataFrames(['UEUp','UEDown','MEPSUp','MEPSDown']) # ttTheory

runCountDataFrames(["RenormUp","RenormDown","FactorUp","FactorDown"]) # ttLHEWeight
runCountDataFrames(["PDFUp","PDFDown","TTXSUp","TWXSUp"]) # ttLHEWeight

runCountDataFrames(['EPtDown','MuPtDown','TauPtDown',""]) # lepEnergy
runCountDataFrames(["JESUp","JESDown","JERUp","JERDown"]) # jetEnergy
runCountDataFrames(["BTagUp","BTagDown","MistagUp","MistagDown"]) # bTagging

counting mu1b ...
counting mu1b ...
counting mu1b ...
counting mu1b ...


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/zihengchen/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-7a86cfb6aa76>", line 11, in <module>
    runCountDataFrames(["RenormUp","RenormDown","FactorUp","FactorDown"]) # ttLHEWeight
  File "<ipython-input-4-7d38c1982ffd>", line 10, in runCountDataFrames
    ps.join()
  File "/Users/zihengchen/anaconda3/lib/python3.6/multiprocessing/process.py", line 124, in join
    res = self._popen.wait(timeout)
  File "/Users/zihengchen/anaconda3/lib/python3.6/multiprocessing/popen_fork.py", line 57, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/Users/zihengchen/anaconda3/lib/python3.6/multiprocessing/popen_fork.py", line 35, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):

KeyboardInterrupt: 

Process Process-2:
Process Process-4:
Process Process-1:
Process Process-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/zihengchen/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/zihengchen/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/zihengchen/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/zihengchen/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/zihengchen/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "<ipython-input-2-3d5c8e38773c>", line 13, in countDataFrames
    acc,accVar = counter.returnAcc()
  File "/Users/zihengchen/anaconda3/lib/python3.6/multiprocessing

## 3. Count uncertainty of Tau acc

In [6]:
# jet misID
df = DFCutter('etau','>1',"mctt").getDataFrame()
np.sum( df.eventWeight*(1+0.002*df.lepton2_pt) )/ np.sum(df.eventWeight)

1.0798576525660608

# 4. Print the Expect Yields Table

In [7]:
ct = DFCounter('e','2b')
selection = 'e4j'

total,totalVar = 0,0


df = DFCutter(selection,ct.nbjet,"mctt").getDataFrame()
n,nVar = np.sum(df.eventWeight),np.sum(df.eventWeight**2)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

df = DFCutter(selection,ct.nbjet,"mct").getDataFrame()
n,nVar = np.sum(df.eventWeight),np.sum(df.eventWeight**2)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

df = DFCutter(selection,ct.nbjet,"mcw").getDataFrame()
n,nVar = np.sum(df.eventWeight),np.sum(df.eventWeight**2)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

df = DFCutter(selection,ct.nbjet,"mcz").getDataFrame()
n,nVar = np.sum(df.eventWeight),np.sum(df.eventWeight**2)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

df = DFCutter(selection,ct.nbjet,"mcdiboson").getDataFrame()
n,nVar = np.sum(df.eventWeight),np.sum(df.eventWeight**2)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

n, nVar = ct.getNFake(selection,ct.nbjet)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

print("{:8.1f} +/- {:4.1f}".format(total,totalVar**0.5) )
df = DFCutter(selection,ct.nbjet,"data2016").getDataFrame()
print("{:8.1f} +/- {:4.1f}".format(np.sum(df.eventWeight),np.sum(df.eventWeight**2)**0.5) )

125501.3 +/- 154.3
  3473.0 +/- 67.3
  1831.1 +/- 61.9
  1088.9 +/- 93.2
    54.4 +/-  1.0
  1489.0 +/- 17.5
133437.7 +/- 202.9
132564.0 +/- 364.1


# 5. Summarize Counts

In [7]:
counts = summaryCounts()
counts.to_pickle( common.getBaseDirectory() + "data/counts/count_summary.pkl")